In [1]:
import tensorflow as tf
import keras
import time

Using TensorFlow backend.


### Drill: 99% MLP
- Try and tune the MLP model from above to achieve 99% accuracy
- How does its runtime compare to the CNN/RNNs?

In [2]:
# import dataset
from keras.datasets import mnist

# model building imports
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# import backend
from keras import backend as K

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
# 60,000 train images, 10,000 test images
# Images are 28*28, array length should be 784

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)

# Convert to float32 for type consistency
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalize values from 0-255 (256 values of pixels) to 1
X_train /= 255
X_test /= 255

# print sample sizes
print('X_train shape: {}\nX_test shape: {}'.format(X_train.shape,
                                                   X_test.shape))

# Convert class vectors to binary class matrices
# Instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

X_train shape: (60000, 784)
X_test shape: (10000, 784)


In [5]:
batch_size = 128
epochs = 25

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Test loss: 0.1143679699817054
Test accuracy: 0.9833


In [11]:
batch_size = 128
epochs = 25

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.1))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='poisson',
              optimizer='adadelta',
              metrics=['accuracy'])

start = time.time()
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('runtime: {} seconds'.format(time.time() - start))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_14 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Test loss: 0.10569524173736572
Test accuracy: 0.9823
runtime: 150.10068893432617 seconds


In [10]:
batch_size = 128
epochs = 20

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adamax',
              metrics=['accuracy'])

start = time.time()
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('runtime: {} seconds'.format(time.time() - start))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Test loss: 0.059238107243273774
Test accuracy: 0.9848
runtime: 125.75391697883606 seconds


In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import time

In [ ]:
def create_mlp(optimizer='rmsprop', loss='categorical_crossentropy'):
    model = Sequential()

    # add layers
    model.add(Dense(64, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.1)) # dropout layers removing features to reduce overfitting

    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.1))

    model.add(Dense(10, activation='softmax')) # set last layer units equal to number of classes

    #model.summary()

    # compile model
    model.compile(
        loss=loss,
        optimizer=optimizer,
        metrics=['accuracy']
    )
    
    return model

In [ ]:
neural_net = KerasClassifier(build_fn=create_mlp, verbose=0)

optimizers = ['adagrad', 'adamax']
loss = ['poisson']
batches = [128]
epochs = [100]

params = dict(optimizer=optimizers, loss=loss, batch_size=batches, epochs=epochs)
grid = GridSearchCV(estimator=neural_net, param_grid=params)

In [ ]:
start_time = time.time()
grid_results = grid.fit(X_train, y_train)

print('params: {}'.format(grid_results.best_params_))
print('score: {}'.format(grid_results.best_score_))
print('gridsearch runtime: {}'.format(time.time() - start_time))

In [ ]:
# compile model
model.compile(
    loss='poisson',
    optimizer='adagrad',
    metrics=['accuracy']
)
# train/evaluate
history = model.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=100,
    verbose=0,
    validation_data=(X_test, y_test)
)
score = model.evaluate(X_test, y_test, verbose=0)
print('test loss: {}'.format(score[0]))
print('test accuracy: {}'.format(score[1]))

In [ ]:
# compile model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adamax',
    metrics=['accuracy']
)
# train/evaluate
history = model.fit(
    X_train,
    y_train,
    batch_size=128,
    epochs=100,
    verbose=0,
    validation_data=(X_test, y_test)
)
score = model.evaluate(X_test, y_test, verbose=0)
print('test loss: {}'.format(score[0]))
print('test accuracy: {}'.format(score[1]))